In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from io import BytesIO

from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.get_bucket('capstone_p')
blob = bucket.blob('yelp_business_merged_preprocessed.csv')
path = "gs://capstone_p/yelp_business_merged_preprocessed.csv"

In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix as cm,classification_report
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
data_com_buisness = pd.read_csv(path,encoding='utf-8')
data_com_buisness =data_com_buisness.head(200000)

In [4]:
from collections import Counter
 
def balance_classes(xs, ys):
##"""Undersample xs, ys to balance classes."""
  freqs = Counter(ys)
  max_allowable = freqs.most_common()[-1][1]
  num_added = {clss: 0 for clss in freqs.keys()}  
  new_ys = []
  new_xs = []
  for i, y in enumerate(ys):
    if num_added[y] < max_allowable:
       new_ys.append(y)
       new_xs.append(xs[i])
       num_added[y] += 1
  return new_xs, new_ys

In [5]:
print(Counter(data_com_buisness['stars_y']))
balanced_x, balanced_y = balance_classes(data_com_buisness['text_1'], data_com_buisness['stars_y'])
print(Counter(balanced_y))

Counter({5: 78617, 4: 52579, 3: 27848, 1: 22068, 2: 18888})
Counter({1: 18888, 2: 18888, 3: 18888, 4: 18888, 5: 18888})


In [17]:
Train_X_balanced, Test_X_balanced, Train_Y_balanced, Test_Y_balanced = model_selection.train_test_split(data_com_buisness['text_1'],data_com_buisness['stars_y'],test_size=0.3)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
vec_transformed = CountVectorizer(min_df=200, max_df=150.0, lowercase=True)
data = vec_transformed.fit_transform(data_com_buisness['text_1'])

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(data)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [23]:
# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=vec_transformed.get_feature_names(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

,idf_weights
food,1.870904
place,1.924126
good,1.959556
great,2.082586
like,2.223008
...,...
overheard,7.902773
chefs,7.902773
sang,7.902773
grandmoth,7.902773


In [28]:
# count matrix
count_vector=vec_transformed.transform(data_com_buisness['text_1'])
 
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [48]:
tf_idf_vector.shape

(200000, 7916)

In [29]:
tf_idf_vector.shape
feature_names = vec_transformed.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]
 
#print the scores
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
brick,0.366105
alway,0.331495
polit,0.311215
beautiful,0.304133
summer,0.291106
...,...
factori,0.000000
fail,0.000000
fair,0.000000
fajita,0.000000


In [30]:
# count matrix
count_vector=vec_transformed.transform(Train_X_balanced)
 
# tf-idf scores
Train_X_Tfidf=tfidf_transformer.transform(count_vector)

# count matrix
count_vector=vec_transformed.transform(Test_X_balanced)
 
# tf-idf scores
Test_X_Tfidf=tfidf_transformer.transform(count_vector)

In [31]:
# count matrix
count_vector=vec_transformed.transform(Test_X_balanced)
 
# tf-idf scores
Test_X_Tfidf=tfidf_transformer.transform(count_vector)

In [32]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(Train_X_Tfidf,Train_Y_balanced)


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=100000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=1, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
y_pred = logreg.predict(Test_X_Tfidf)
print(cm(Test_Y_balanced, y_pred))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y_balanced, y_pred))
print("log reg  Accuracy Score ->",metrics.accuracy_score(Test_Y_balanced, y_pred))

[[ 4806   920   390   235   301]
 [ 1536  1616  1435   707   366]
 [  554  1013  2931  3006   991]
 [  188   229  1281  7622  6400]
 [  176    89   283  3513 19412]]


Classification Report 
              precision    recall  f1-score   support

           1       0.66      0.72      0.69      6652
           2       0.42      0.29      0.34      5660
           3       0.46      0.35      0.40      8495
           4       0.51      0.48      0.49     15720
           5       0.71      0.83      0.76     23473

    accuracy                           0.61     60000
   macro avg       0.55      0.53      0.54     60000
weighted avg       0.59      0.61      0.59     60000

log reg  Accuracy Score -> 0.60645


In [35]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(Train_X_Tfidf,Train_Y_balanced)
y_pred_svc1 = model.predict(Test_X_Tfidf)
print(cm(Test_Y_balanced, y_pred_svc1))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y_balanced, y_pred_svc1))
print("log reg  Accuracy Score ->",metrics.accuracy_score(Test_Y_balanced, y_pred_svc1))

[[ 5016   780   334   205   317]
 [ 1741  1459  1343   713   404]
 [  634   928  2718  3133  1082]
 [  229   199  1116  7453  6723]
 [  175    78   240  3220 19760]]


Classification Report 
              precision    recall  f1-score   support

           1       0.64      0.75      0.69      6652
           2       0.42      0.26      0.32      5660
           3       0.47      0.32      0.38      8495
           4       0.51      0.47      0.49     15720
           5       0.70      0.84      0.76     23473

    accuracy                           0.61     60000
   macro avg       0.55      0.53      0.53     60000
weighted avg       0.58      0.61      0.59     60000

log reg  Accuracy Score -> 0.6067666666666667


In [37]:
from sklearn.feature_extraction.text import CountVectorizer
vec_transformed = CountVectorizer(min_df=200, max_df=150.0, lowercase=True,ngram_range=(1, 2), analyzer='word')
data = vec_transformed.fit_transform(data_com_buisness['text_1'])

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(data)

df_idf = pd.DataFrame(tfidf_transformer.idf_, index=vec_transformed.get_feature_names(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

,idf_weights
food,1.870904
place,1.924126
good,1.959556
great,2.082586
like,2.223008
...,...
extra star,7.902773
huge plus,7.902773
vietnames restaur,7.902773
boys,7.902773


In [38]:
# count matrix
count_vector=vec_transformed.transform(data_com_buisness['text_1'])
 
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [39]:
feature_names = vec_transformed.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]
 
#print the scores
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
enjoy time,0.321210
alway enjoy,0.312027
brick,0.294162
never disappoint,0.292690
alway,0.266353
...,...
fortun,0.000000
forth,0.000000
former,0.000000
formal,0.000000


In [40]:
count_vector=vec_transformed.transform(Train_X_balanced)
 
# tf-idf scores
Train_X_Tfidf=tfidf_transformer.transform(count_vector)

In [41]:
# count matrix
count_vector=vec_transformed.transform(Test_X_balanced)
 
# tf-idf scores
Test_X_Tfidf=tfidf_transformer.transform(count_vector)

[[ 4924   914   351   202   261]
 [ 1625  1634  1393   663   345]
 [  569  1063  2998  2936   929]
 [  193   248  1258  7671  6350]
 [  149    84   300  3418 19522]]


Classification Report 
              precision    recall  f1-score   support

           1       0.66      0.74      0.70      6652
           2       0.41      0.29      0.34      5660
           3       0.48      0.35      0.41      8495
           4       0.52      0.49      0.50     15720
           5       0.71      0.83      0.77     23473

    accuracy                           0.61     60000
   macro avg       0.56      0.54      0.54     60000
weighted avg       0.59      0.61      0.60     60000

log reg  Accuracy Score -> 0.6124833333333334


In [43]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(Train_X_Tfidf,Train_Y_balanced)
y_pred = logreg.predict(Test_X_Tfidf)
print(cm(Test_Y_balanced, y_pred))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y_balanced, y_pred))
print("log reg  Accuracy Score ->",metrics.accuracy_score(Test_Y_balanced, y_pred))

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[[ 4632  1086   450   240   244]
 [ 1477  1737  1475   665   306]
 [  530  1144  3128  2826   867]
 [  222   298  1480  7671  6049]
 [  168    99   331  3709 19166]]


Classification Report 
              precision    recall  f1-score   support

           1       0.66      0.70      0.68      6652
           2       0.40      0.31      0.35      5660
           3       0.46      0.37      0.41      8495
           4       0.51      0.49      0.50     15720
           5       0.72      0.82      0.77     23473

    accuracy                           0.61     60000
   macro avg       0.55      0.54      0.54     60000
weighted avg       0.59      0.61      0.60     60000

log reg  Accuracy Score -> 0.6055666666666667


In [46]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=1.0, max_features=200, max_depth=50, random_state=0)
gb_clf.fit(Train_X_Tfidf,Train_Y_balanced)

y_pred = gb_clf.predict(Test_X_Tfidf)
print(cm(Test_Y_balanced, y_pred))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y_balanced, y_pred))
print("grad Accuracy Score ->",metrics.accuracy_score(Test_Y_balanced, y_pred))


GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=1.0, loss='deviance', max_depth=50,
                           max_features=200, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=20,
                           n_iter_no_change=None, presort='auto',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

[[ 2710  1170   894   864  1014]
 [  965  1050  1340  1426   879]
 [  604   968  2097  3332  1494]
 [  495   682  1766  6463  6314]
 [  568   520  1004  4911 16470]]


Classification Report 
              precision    recall  f1-score   support

           1       0.51      0.41      0.45      6652
           2       0.24      0.19      0.21      5660
           3       0.30      0.25      0.27      8495
           4       0.38      0.41      0.40     15720
           5       0.63      0.70      0.66     23473

    accuracy                           0.48     60000
   macro avg       0.41      0.39      0.40     60000
weighted avg       0.47      0.48      0.47     60000

log reg  Accuracy Score -> 0.47983333333333333


In [62]:
from sklearn.feature_extraction.text import CountVectorizer
vec_transformed = CountVectorizer(min_df=400, max_df=290.0, lowercase=True,ngram_range=(1, 3), analyzer='word')
data = vec_transformed.fit_transform(data_com_buisness['text_1'])

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(data)

df_idf = pd.DataFrame(tfidf_transformer.idf_, index=vec_transformed.get_feature_names(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

,idf_weights
food,1.870904
place,1.924126
good,1.959556
great,2.082586
like,2.223008
...,...
promptly,7.212116
afterwards,7.212116
underwhelm,7.212116
wait minut,7.212116


In [63]:
# count matrix
count_vector=vec_transformed.transform(data_com_buisness['text_1'])
 
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [64]:
tf_idf_vector.shape

(200000, 4090)

In [65]:

feature_names = vec_transformed.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]
 
#print the scores
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
brick,0.328993
never disappoint,0.327348
alway,0.297892
staff alway,0.292074
polit,0.279667
...,...
food look,0.000000
food love,0.000000
food menu,0.000000
food ok,0.000000


In [66]:
# count matrix
count_vector=vec_transformed.transform(Train_X_balanced)
 
# tf-idf scores
Train_X_Tfidf=tfidf_transformer.transform(count_vector)

# count matrix
count_vector=vec_transformed.transform(Test_X_balanced)
 
# tf-idf scores
Test_X_Tfidf=tfidf_transformer.transform(count_vector)

In [67]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(Train_X_Tfidf,Train_Y_balanced)
y_pred_svc1 = model.predict(Test_X_Tfidf)
print(cm(Test_Y_balanced, y_pred_svc1))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y_balanced, y_pred_svc1))
print("log reg  Accuracy Score ->",metrics.accuracy_score(Test_Y_balanced, y_pred_svc1))

[[ 5077   765   337   192   281]
 [ 1699  1527  1377   682   375]
 [  626   946  2884  3047   992]
 [  206   222  1120  7642  6530]
 [  167    77   245  3255 19729]]


Classification Report 
              precision    recall  f1-score   support

           1       0.65      0.76      0.70      6652
           2       0.43      0.27      0.33      5660
           3       0.48      0.34      0.40      8495
           4       0.52      0.49      0.50     15720
           5       0.71      0.84      0.77     23473

    accuracy                           0.61     60000
   macro avg       0.56      0.54      0.54     60000
weighted avg       0.59      0.61      0.60     60000

log reg  Accuracy Score -> 0.6143166666666666


In [61]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(Train_X_Tfidf,Train_Y_balanced)
y_pred_svc1 = model.predict(Test_X_Tfidf)
print(cm(Test_Y_balanced, y_pred_svc1))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y_balanced, y_pred_svc1))
print("log reg  Accuracy Score ->",metrics.accuracy_score(Test_Y_balanced, y_pred_svc1))

[[ 5077   765   337   192   281]
 [ 1699  1527  1377   682   375]
 [  626   946  2884  3047   992]
 [  206   222  1120  7642  6530]
 [  167    77   245  3255 19729]]


Classification Report 
              precision    recall  f1-score   support

           1       0.65      0.76      0.70      6652
           2       0.43      0.27      0.33      5660
           3       0.48      0.34      0.40      8495
           4       0.52      0.49      0.50     15720
           5       0.71      0.84      0.77     23473

    accuracy                           0.61     60000
   macro avg       0.56      0.54      0.54     60000
weighted avg       0.59      0.61      0.60     60000

log reg  Accuracy Score -> 0.6143166666666666


In [54]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(Train_X_Tfidf,Train_Y_balanced)
y_pred = logreg.predict(Test_X_Tfidf)
print(cm(Test_Y_balanced, y_pred))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y_balanced, y_pred))
print("log reg  Accuracy Score ->",metrics.accuracy_score(Test_Y_balanced, y_pred))

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[[ 4638  1100   439   224   251]
 [ 1482  1731  1469   671   307]
 [  546  1135  3129  2829   856]
 [  215   302  1484  7704  6015]
 [  172   100   353  3736 19112]]


Classification Report 
              precision    recall  f1-score   support

           1       0.66      0.70      0.68      6652
           2       0.40      0.31      0.35      5660
           3       0.46      0.37      0.41      8495
           4       0.51      0.49      0.50     15720
           5       0.72      0.81      0.76     23473

    accuracy                           0.61     60000
   macro avg       0.55      0.54      0.54     60000
weighted avg       0.59      0.61      0.59     60000

log reg  Accuracy Score -> 0.6052333333333333


In [68]:
from sklearn.feature_extraction.text import CountVectorizer
vec_transformed = CountVectorizer(min_df=400, max_df=150.0, lowercase=True,ngram_range=(1, 4), analyzer='word')
data = vec_transformed.fit_transform(data_com_buisness['text_1'])

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(data)

df_idf = pd.DataFrame(tfidf_transformer.idf_, index=vec_transformed.get_feature_names(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

,idf_weights
food,1.870904
place,1.924126
good,1.959556
great,2.082586
like,2.223008
...,...
wait minut,7.212116
still wait,7.212116
underwhelm,7.212116
promptly,7.212116


In [69]:
count_vector=vec_transformed.transform(data_com_buisness['text_1'])
 
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [70]:
# count matrix
count_vector=vec_transformed.transform(Train_X_balanced)
 
# tf-idf scores
Train_X_Tfidf=tfidf_transformer.transform(count_vector)

# count matrix
count_vector=vec_transformed.transform(Test_X_balanced)
 
# tf-idf scores
Test_X_Tfidf=tfidf_transformer.transform(count_vector)

In [71]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(Train_X_Tfidf,Train_Y_balanced)
y_pred_svc1 = model.predict(Test_X_Tfidf)
print(cm(Test_Y_balanced, y_pred_svc1))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y_balanced, y_pred_svc1))
print("log reg  Accuracy Score ->",metrics.accuracy_score(Test_Y_balanced, y_pred_svc1))

[[ 5077   765   338   192   280]
 [ 1700  1527  1377   681   375]
 [  625   946  2882  3049   993]
 [  206   221  1120  7644  6529]
 [  167    76   246  3256 19728]]


Classification Report 
              precision    recall  f1-score   support

           1       0.65      0.76      0.70      6652
           2       0.43      0.27      0.33      5660
           3       0.48      0.34      0.40      8495
           4       0.52      0.49      0.50     15720
           5       0.71      0.84      0.77     23473

    accuracy                           0.61     60000
   macro avg       0.56      0.54      0.54     60000
weighted avg       0.59      0.61      0.60     60000

log reg  Accuracy Score -> 0.6143


In [72]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(Train_X_Tfidf,Train_Y_balanced)
y_pred = logreg.predict(Test_X_Tfidf)
print(cm(Test_Y_balanced, y_pred))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y_balanced, y_pred))
print("log reg  Accuracy Score ->",metrics.accuracy_score(Test_Y_balanced, y_pred))

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[[ 4866   909   390   215   272]
 [ 1545  1653  1471   649   342]
 [  544  1014  3064  2986   887]
 [  183   240  1284  7803  6210]
 [  159    78   294  3517 19425]]


Classification Report 
              precision    recall  f1-score   support

           1       0.67      0.73      0.70      6652
           2       0.42      0.29      0.35      5660
           3       0.47      0.36      0.41      8495
           4       0.51      0.50      0.51     15720
           5       0.72      0.83      0.77     23473

    accuracy                           0.61     60000
   macro avg       0.56      0.54      0.55     60000
weighted avg       0.60      0.61      0.60     60000

log reg  Accuracy Score -> 0.6135166666666667


In [74]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=1.0, max_features=900, max_depth=150, random_state=0)
gb_clf.fit(Train_X_Tfidf,Train_Y_balanced)

y_pred = gb_clf.predict(Test_X_Tfidf)
print(cm(Test_Y_balanced, y_pred))
print('\n')
print("Classification Report ")
print(classification_report(Test_Y_balanced, y_pred))
print("grad Accuracy Score ->",metrics.accuracy_score(Test_Y_balanced, y_pred))


[[ 3517   943   520   866   806]
 [ 1157  1117  1113  1424   849]
 [  605  1108  1900  3330  1552]
 [  404   732  1583  6328  6673]
 [  437   500   779  4114 17643]]


Classification Report 
              precision    recall  f1-score   support

           1       0.57      0.53      0.55      6652
           2       0.25      0.20      0.22      5660
           3       0.32      0.22      0.26      8495
           4       0.39      0.40      0.40     15720
           5       0.64      0.75      0.69     23473

    accuracy                           0.51     60000
   macro avg       0.44      0.42      0.43     60000
weighted avg       0.49      0.51      0.49     60000

grad Accuracy Score -> 0.5084166666666666
